In [1]:
# Celda 1: Reinstalar
!pip uninstall -y transformers
!pip install -q transformers==4.38.0 safetensors pillow
!pip install accelerate==0.27.0
!pip install -q jiwer rouge-score bert-score

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.12.0
    Uninstalling accelerate-1.12.0:
      Successfully uninstalled accelerate-1.12.0
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61

In [ ]:
import transformers
import accelerate
print(f" Transformers: {transformers.__version__}")
print(f" Accelerate: {accelerate.__version__}")

 Transformers: 4.38.0
 Accelerate: 0.27.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
OCR_path = '/content/drive/MyDrive/Imagenes Proyecto/OCR'
IC_path = '/content/drive/MyDrive/Imagenes Proyecto/IC'
annotations_path = '/content/drive/MyDrive/Imagenes Proyecto/annotations.json'
results_path = '/content/drive/MyDrive/Imagenes Proyecto/results_florence2.json'

In [4]:
import json
# Cargar anotaciones
with open(annotations_path, 'r', encoding='utf-8') as f:
    annotations_raw = json.load(f)

print(f"Anotaciones cargadas: {len(annotations_raw)} imágenes")

Anotaciones cargadas: 30 imágenes


In [5]:
# Convertir annotations al formato esperado
annotations = []
for img_name, img_data in annotations_raw.items():
    # Procesar OCR
    if img_data['ocr']:  # Si tiene anotaciones OCR
        annotations.append({
            'image_path': img_name,
            'task': 'OCR',
            'annotations': img_data['ocr']
        })

    # Procesar IC
    if any(img_data['ic'].values()):  # Si tiene al menos una descripción IC
        ic_annotations = [
            img_data['ic']['basic'],
            img_data['ic']['intermediate'],
            img_data['ic']['advanced']
        ]
        # Filtrar descripciones vacías
        ic_annotations = [desc for desc in ic_annotations if desc.strip()]

        if ic_annotations:
            annotations.append({
                'image_path': img_name,
                'task': 'IC',
                'annotations': ic_annotations
            })

print(f"Total de tareas a procesar: {len(annotations)}")
print(f"  - OCR: {sum(1 for a in annotations if a['task'] == 'OCR')}")
print(f"  - IC: {sum(1 for a in annotations if a['task'] == 'IC')}")


Total de tareas a procesar: 30
  - OCR: 15
  - IC: 15


In [2]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image

In [5]:
print(torch.__version__)

2.9.0+cu126


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

model_id = "microsoft/Florence-2-base-ft"  

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch_dtype  # ← Cambiar de "eager" a "sdpa"
).to(device)

processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [ ]:
!pip install -q jiwer rouge-score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.1 MB/s eta 0:00:00


In [ ]:
import os
from PIL import Image
import time
from jiwer import wer, cer
from rouge_score import rouge_scorer
from bert_score import score as bert_score

In [ ]:
# Función para ejecutar Florence-2
def run_florence2(image_path, task_prompt):
    """Ejecuta Florence-2 con una imagen y prompt específico"""
    image = Image.open(image_path).convert('RGB')

    inputs = processor(text=task_prompt, images=image, return_tensors="pt").to(device, torch_dtype)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3,
        do_sample=False
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [ ]:
# Definir prompts para OCR
ocr_prompts = [
    "<OCR>",  # Prompt básico de Florence-2
    "<OCR_WITH_REGION>"  # Prompt con regiones
]

# Definir prompts para IC (Image Captioning)
ic_prompts = {
    "basic": "<CAPTION>",  # Descripción básica
    "detailed": "<DETAILED_CAPTION>",  # Descripción detallada
    "more_detailed": "<MORE_DETAILED_CAPTION>"  # Descripción muy detallada
}

In [ ]:
# Función para calcular métricas OCR
def calculate_ocr_metrics(reference, hypothesis):
    """Calcula CER y WER"""
    if not reference or not hypothesis:
        return {"CER": 1.0, "WER": 1.0}

    cer_score = cer(reference, hypothesis)
    wer_score = wer(reference, hypothesis)

    return {
        "CER": round(cer_score, 4),
        "WER": round(wer_score, 4)
    }

# Función para calcular métricas IC
def calculate_ic_metrics(references, hypothesis):
    """Calcula ROUGE-L y BERTScore"""
    if not references or not hypothesis:
        return {"ROUGE-L": 0.0, "BERTScore": 0.0}

    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, hypothesis)['rougeL'].fmeasure for ref in references]
    rouge_l = max(rouge_scores)  # Tomar el mejor score entre las referencias

    # BERTScore
    P, R, F1 = bert_score([hypothesis] * len(references), references, lang='en', verbose=False)
    bert_score_val = F1.max().item()  # Tomar el mejor F1

    return {
        "ROUGE-L": round(rouge_l, 4),
        "BERTScore": round(bert_score_val, 4)
    }

# Estructura para guardar resultados
results = []

In [ ]:
# PROCESAMIENTO IC (IMAGE CAPTIONING)

print("\n" + "="*60)
print("BLOQUE 1: PROCESAMIENTO IC")
print("="*60)

# Definir prompts CORRECTOS para IC
ic_prompts = [
    "<CAPTION>",
    "<DETAILED_CAPTION>",
    "<MORE_DETAILED_CAPTION>"
]

ic_annotations = [a for a in annotations if a['task'] == 'IC']

for idx, img_data in enumerate(ic_annotations):
    image_path = img_data['image_path']

    print(f"\n[{idx+1}/{len(ic_annotations)}] Procesando: {image_path} (IC)")

    full_path = os.path.join(IC_path, os.path.basename(image_path))

    if not os.path.exists(full_path):
        print(f"  Imagen no encontrada: {full_path}")
        continue

    prompt_outputs = []

    for i, prompt in enumerate(ic_prompts):
        print(f"  Ejecutando prompt {i+1}/{len(ic_prompts)}: {prompt}...")

        start_time = time.time()
        try:
            result = run_florence2(full_path, prompt)
            execution_time = time.time() - start_time

            # Extraer caption
            output_text = result.get(prompt, '')

            print(f"    Output: {output_text[:100]}...")

            # Calcular métricas IC
            references = img_data['annotations']
            scores = calculate_ic_metrics(references, output_text)
            metrics = {
                "score_a": scores["ROUGE-L"],
                "score_b": scores["BERTScore"]
            }

            prompt_outputs.append({
                "prompt": prompt,  # ← Guarda el prompt correcto
                "output": output_text,
                "execution_time": round(execution_time, 2),
                "scores": metrics
            })

        except Exception as e:
            print(f"    Error: {str(e)}")
            prompt_outputs.append({
                "prompt": prompt,
                "output": "",
                "execution_time": 0,
                "scores": {"score_a": 0.0, "score_b": 0.0},
                "error": str(e)
            })

    # Agregar resultado IC
    results.append({
        "image_path": image_path,
        "model": "Florence-2-base-ft",
        "task": "IC",
        "annotations": img_data['annotations'],
        "prompt_outputs": prompt_outputs
    })

print("\n BLOQUE IC COMPLETADO")


BLOQUE 1: PROCESAMIENTO IC

[1/15] Procesando: 01.jpg (IC)
  Ejecutando prompt 1/3: <CAPTION>...
    Output: A 500 pesos coin from 2002 sits on a table....


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Ejecutando prompt 2/3: <DETAILED_CAPTION>...
    Output: In this image we can see a coin placed on the surface....


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Ejecutando prompt 3/3: <MORE_DETAILED_CAPTION>...
    Output: A 500 Pesos coin from 2002 is sitting on a table. The coin is gold in color and has a floral design ...


...

  Ejecutando prompt 2/3: <DETAILED_CAPTION>...
    Output: In this image we can see two persons sitting on the motorbike....


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Ejecutando prompt 3/3: <MORE_DETAILED_CAPTION>...
    Output: A man is sitting on a motorcycle. He is wearing a black jacket and blue jeans. He has a black helmet...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 BLOQUE IC COMPLETADO


In [ ]:
print("\n" + "="*50)
print("Guardando resultados IC...")

# Guardar solo IC en archivo separado
ic_results_path = '/content/drive/MyDrive/Imagenes Proyecto/results_florence2_IC_only.json'
ic_only_results = [r for r in results if r['task'] == 'IC']

with open(ic_results_path, 'w', encoding='utf-8') as f:
    json.dump(ic_only_results, f, indent=2, ensure_ascii=False)

print(f"Resultados IC guardados en: {ic_results_path}")
print(f"Total de imágenes IC procesadas: {len(ic_only_results)}")


Guardando resultados IC...
Resultados IC guardados en: /content/drive/MyDrive/Imagenes Proyecto/results_florence2_IC_only.json
Total de imágenes IC procesadas: 15


In [ ]:
# BLOQUE 2: PROCESAMIENTO OCR

print("\n" + "="*60)
print("BLOQUE 2: PROCESAMIENTO OCR")
print("="*60)

ocr_annotations = [a for a in annotations if a['task'] == 'OCR']

for idx, img_data in enumerate(ocr_annotations):
    image_path = img_data['image_path']

    print(f"\n[{idx+1}/{len(ocr_annotations)}] Procesando: {image_path} (OCR)")

    full_path = os.path.join(OCR_path, os.path.basename(image_path))

    if not os.path.exists(full_path):
        print(f"  Imagen no encontrada: {full_path}")
        continue

    prompt_outputs = []

    for i, prompt in enumerate(ocr_prompts):
        print(f"  Ejecutando prompt {i+1}/{len(ocr_prompts)}: {prompt}...")

        start_time = time.time()
        try:
            result = run_florence2(full_path, prompt)
            execution_time = time.time() - start_time

            # DEBUG: Ver qué devuelve
            print(f" DEBUG - Tipo: {type(result)}")

            # Extraer texto con manejo robusto
            output_text = ""

            if isinstance(result, dict):
                # Si es diccionario
                if prompt in result:
                    content = result[prompt]

                    if isinstance(content, dict):
                        # Buscar 'text' o 'labels'
                        output_text = content.get('text', '')
                        if not output_text and 'labels' in content:
                            labels = content.get('labels', [])
                            output_text = ' '.join(labels) if isinstance(labels, list) else str(labels)
                    elif isinstance(content, str):
                        output_text = content
                    elif isinstance(content, list):
                        output_text = ' '.join(str(item) for item in content)
                    else:
                        output_text = str(content)
            elif isinstance(result, str):
                # Si es string directo
                output_text = result
            else:
                output_text = str(result)

            print(f" Output: {output_text[:100]}...")

            # Calcular métricas OCR
            scores = calculate_ocr_metrics(img_data['annotations'], output_text)
            metrics = {
                "score_a": scores["CER"],
                "score_b": scores["WER"]
            }
            print(f" CER: {scores['CER']:.4f} | WER: {scores['WER']:.4f}")

            prompt_outputs.append({
                "prompt": prompt,
                "output": output_text,
                "execution_time": round(execution_time, 2),
                "scores": metrics
            })

        except Exception as e:
            print(f" Error: {str(e)}")
            import traceback
            print(f" Traceback: {traceback.format_exc()}")
            prompt_outputs.append({
                "prompt": prompt,
                "output": "",
                "execution_time": 0,
                "scores": {"score_a": 0.0, "score_b": 0.0},
                "error": str(e)
            })

    # Agregar resultado OCR
    results.append({
        "image_path": image_path,
        "model": "Florence-2-base-ft",
        "task": "OCR",
        "annotations": img_data['annotations'],
        "prompt_outputs": prompt_outputs
    })

print("\n BLOQUE OCR COMPLETADO")


BLOQUE 2: PROCESAMIENTO OCR

[1/15] Procesando: 20220101_035959.jpg (OCR)
  Ejecutando prompt 1/2: <OCR>...
 Error: 'NoneType' object has no attribute 'shape'
 Traceback: Traceback (most recent call last):
  File "/tmp/ipython-input-3382759189.py", line 29, in <cell line: 0>
    result = run_florence2(full_path, prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-810634766.py", line 8, in run_florence2
    generated_ids = model.generate(
                    ^^^^^^^^^^^^^^^
  File "/root/.cache/huggingface/modules/transformers_modules/microsoft/Florence_hyphen_2_hyphen_base_hyphen_ft/f6c1a25888ffc1d945ee8a1a77ac833c7303d46e/modeling_florence2.py", line 2795, in generate
    return self.language_model.generate(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dis

KeyboardInterrupt: 

In [ ]:
# BLOQUE 2: PROCESAMIENTO OCR

print("\n" + "="*60)
print("BLOQUE 2: PROCESAMIENTO OCR")
print("="*60)

ocr_annotations = [a for a in annotations if a['task'] == 'OCR']

for idx, img_data in enumerate(ocr_annotations):
    image_path = img_data['image_path']

    print(f"\n[{idx+1}/{len(ocr_annotations)}] Procesando: {image_path} (OCR)")

    full_path = os.path.join(OCR_path, os.path.basename(image_path))

    if not os.path.exists(full_path):
        print(f"  Imagen no encontrada: {full_path}")
        continue

    prompt_outputs = []

    for i, prompt in enumerate(ocr_prompts):
        print(f"  Ejecutando prompt {i+1}/{len(ocr_prompts)}...")

        start_time = time.time()
        try:
            result = run_florence2(full_path, prompt)
            execution_time = time.time() - start_time

            # Extraer el texto del resultado (OCR)
            output_text = result.get(prompt, {}).get('text', '')

            print(f"    Output: {output_text[:100]}...")

            # Calcular métricas OCR
            reference = img_data['annotations'][0] if img_data['annotations'] else ""
            scores = calculate_ocr_metrics(img_data['annotations'], output_text)
            metrics = {
                "score_a": scores["CER"],
                "score_b": scores["WER"]
            }

            prompt_outputs.append({
                "prompt": prompt,
                "output": output_text,
                "execution_time": round(execution_time, 2),
                "scores": metrics
            })

        except Exception as e:
            print(f"    Error: {str(e)}")
            prompt_outputs.append({
                "prompt": prompt,
                "output": "",
                "execution_time": 0,
                "scores": {"score_a": 0.0, "score_b": 0.0},
                "error": str(e)
            })

    # Agregar resultado OCR
    results.append({
        "image_path": image_path,
        "model": "Florence-2-base-ft",
        "task": "OCR",
        "annotations": img_data['annotations'],
        "prompt_outputs": prompt_outputs
    })

print("\n BLOQUE OCR COMPLETADO")


BLOQUE 2: PROCESAMIENTO OCR

[1/15] Procesando: 20220101_035959.jpg (OCR)
  Ejecutando prompt 1/2...
    Error: 'str' object has no attribute 'get'
  Ejecutando prompt 2/2...


KeyboardInterrupt: 

In [ ]:
print("\n" + "="*50)
print("Guardando resultados...")
with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Resultados guardados en: {results_path}")
print(f"Total de imágenes procesadas: {len(results)}")

# Mostrar resumen
print("\n" + "="*50)
print("RESUMEN DE RESULTADOS")
print("="*50)

ocr_results = [r for r in results if r['task'] == 'OCR']
ic_results = [r for r in results if r['task'] == 'IC']

if ocr_results:
    print(f"\nOCR - {len(ocr_results)} imágenes:")
    avg_cer = sum([p['scores']['score_a'] for r in ocr_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ocr_results])
    avg_wer = sum([p['scores']['score_b'] for r in ocr_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ocr_results])
    print(f"  CER promedio: {avg_cer:.4f}")
    print(f"  WER promedio: {avg_wer:.4f}")

if ic_results:
    print(f"\nIC - {len(ic_results)} imágenes:")
    avg_rouge = sum([p['scores']['score_a'] for r in ic_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ic_results])
    avg_bert = sum([p['scores']['score_b'] for r in ic_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ic_results])
    print(f"  ROUGE-L promedio: {avg_rouge:.4f}")
    print(f"  BERTScore promedio: {avg_bert:.4f}")

print("\n Proceso completado!")


Guardando resultados...
Resultados guardados en: /content/drive/MyDrive/Imagenes Proyecto/results_florence2.json
Total de imágenes procesadas: 30

RESUMEN DE RESULTADOS

IC - 30 imágenes:
  ROUGE-L promedio: 0.1423
  BERTScore promedio: 0.8544

 Proceso completado!


In [ ]:
import json
# Guardar resultados
print("\n" + "="*50)
print("Guardando resultados...")
with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Resultados guardados en: {results_path}")
print(f"Total de imágenes procesadas: {len(results)}")



Guardando resultados...
Resultados guardados en: /content/drive/MyDrive/Imagenes Proyecto/results_florence2.json
Total de imágenes procesadas: 30


In [ ]:
# Mostrar resumen de resultados
print("\n" + "="*50)
print("RESUMEN DE RESULTADOS")
print("="*50)

ocr_results = [r for r in results if r['task'] == 'OCR']
ic_results = [r for r in results if r['task'] == 'IC']

if ocr_results:
    print(f"\nOCR - {len(ocr_results)} imágenes:")
    avg_cer = sum([p['scores']['score_a'] for r in ocr_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ocr_results])
    avg_wer = sum([p['scores']['score_b'] for r in ocr_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ocr_results])
    print(f"  CER promedio: {avg_cer:.4f}")
    print(f"  WER promedio: {avg_wer:.4f}")

if ic_results:
    print(f"\nIC - {len(ic_results)} imágenes:")
    avg_rouge = sum([p['scores']['score_a'] for r in ic_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ic_results])
    avg_bert = sum([p['scores']['score_b'] for r in ic_results for p in r['prompt_outputs']]) / sum([len(r['prompt_outputs']) for r in ic_results])
    print(f"  ROUGE-L promedio: {avg_rouge:.4f}")
    print(f"  BERTScore promedio: {avg_bert:.4f}")

print("\n Proceso completado!")


RESUMEN DE RESULTADOS

OCR - 15 imágenes:
  CER promedio: 0.5000
  WER promedio: 0.5000

IC - 15 imágenes:
  ROUGE-L promedio: 0.2846
  BERTScore promedio: 0.8946

 Proceso completado!
